# 模型评估

评估一下之前的LeNet和AlexNet模型，看看效果如何。

In [1]:
import torch
import torch.nn as nn
import numpy as np
from models import LeNet,AlexNet # 注：LeNet的数据不用transform，AlexNet的数据要transform
import torchvision.transforms as transforms
train_labels = torch.load('../data/train_labels.pt')
train_data = torch.load('../data/train_data.pt')
test_labels = torch.load('../data/test_labels.pt')
test_data = torch.load('../data/test_data.pt')

In [2]:
def eval_train_and_test_acc(model):
    
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()
    with torch.no_grad():
        y_pred = model(train_data.to(device))
        y_pred = torch.softmax(y_pred, dim=1)  # dim=1表示对每一行进行softmax
        y_pred = torch.argmax(y_pred, dim=1)
        accuracy = torch.sum(y_pred == train_labels.to(device)).item() / len(train_labels)
    print(f"Train Accuracy: {accuracy:.2f}")
    with torch.no_grad():
        y_pred = model(test_data.to(device))
        y_pred = torch.softmax(y_pred, dim=1)
        y_pred = torch.argmax(y_pred, dim=1)
        accuracy = torch.sum(y_pred == test_labels.to(device)).item() / len(test_labels)
    print(f"Test Accuracy: {accuracy:.2f}")

## 评估LeNet

In [3]:
model = LeNet()
# 在测试集上评估模型的准确率
model.load_state_dict(torch.load("model_LeNet.pth"))
eval_train_and_test_acc(model)

Train Accuracy: 0.55
Test Accuracy: 0.49


## 评估AlexNet

In [5]:
import os
model = AlexNet()
print('==> Resuming from checkpoint..')
assert os.path.isdir('checkpoint_colab'), 'Error: no checkpoint_colab directory found!'
checkpoint = torch.load('./checkpoint_colab/ckpt.pth', map_location=torch.device('cpu'))

==> Resuming from checkpoint..


In [6]:
model.load_state_dict(checkpoint['net'])
best_acc = checkpoint['acc']
start_epoch = checkpoint['epoch']

In [7]:
best_acc

69.9

In [8]:
start_epoch

19

```python
# 用原数据评估是错的，因为我们是在归一化的数据上训练的
# model = AlexNet()
# model.load_state_dict(checkpoint['net'])
# eval_train_and_test_acc(model)
# Train Accuracy: 0.35
# Test Accuracy: 0.35
```

In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
# 只需要归一化
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# 加载 CIFAR10 数据集
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,download=True, transform=transform_test)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128,shuffle=False, num_workers=2)
testset = torchvision.datasets.CIFAR10(root='./data', train=False,download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=128,shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [15]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for data in trainloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print('Accuracy of the network on the 50000 train images: %d %%' % (100 * correct / total))

with torch.no_grad():
    correct = 0
    total = 0
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

Accuracy of the network on the 50000 train images: 72 %
Accuracy of the network on the 10000 test images: 69 %


In [18]:
torch.max(outputs.data, 1)

torch.return_types.max(
values=tensor([ 3.1295,  3.3430,  1.9316,  4.0670, 10.7482,  4.0784,  4.8922,  3.4887,
         3.6127,  2.9611,  2.5348,  2.1729,  2.1153,  6.0943,  1.8651, 11.7545]),
indices=tensor([7, 5, 0, 0, 8, 4, 7, 0, 3, 3, 3, 8, 5, 5, 4, 7]))